<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EA%B5%90%EA%B3%BC%EC%84%B8%ED%8A%B9_%EC%A0%84%EC%B2%98%EB%A6%ACRPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이타 전처리 방법
1. 원데이타를 srecord_raw_306.csv로 다른이름 저장
2. srecord_raw_306.csv 파일을 열어 불필요한 header 윗부분 삭제
3. 열(변수명)이름의 공백을 제거 (ex: '이 름'을 '이름'으로)
4. '세부능력 및 특기사항' 열의 변수명을 '세특'으로 짧게 변경
5. 변경내용을 저장
6. 구글 드라이브에 srecord_raw_306.csv파일을 올려놓기(upload)
7. Colab에서 srecord_raw_306.csv 파일을 불러오기

In [1]:
import pandas as pd

# Read the Excel file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/python/dungchon/srecord_306_raw.csv', encoding='cp949')

# remove 이름과 세특 열 사이의 불필요한 'Unnamed: 4' column
df = df.drop('Unnamed: 4', axis=1)

# remove rows with only NaN values
df = df.dropna(how='all')  

# fill missing values in '이름' column with previous value
df['이름'] = df['이름'].fillna(method='ffill')

# fill missing values in '세특' column with next value
df['세특'] = df.fillna(method='bfill', axis=1)['세특']

# replace them with empty strings before extracting the first five characters:
df['세특'] = df['세특'].fillna('').astype(str)
df['과목'] = df['과목'].fillna(df['세특'].str[:5])

# replace missing values in '과목' column with the first to fifth characters of '세특' column
df['과목'] = df.apply(lambda x: x['세특'][0:5] if pd.isna(x['과목']) else x['과목'], axis=1)

# replace NaN values in 'Unnamed: 12' column with empty string
df['Unnamed: 12'] = df['Unnamed: 12'].fillna('')

# Remove rows containing '등촌고등학교'
df = df[~df['Unnamed: 12'].str.contains('등촌고등학교')]

# sort data by '이름' column
df = df.sort_values('이름')

# fill missing values in '이름' column with previous value
df['이름'] = df['이름'].fillna(method='ffill')

# Write the cleaned DataFrame back to Excel
df.to_excel('/content/drive/MyDrive/python/dungchon/result_d/srecord_RPA.xlsx', index=False)
